In [ ]:
#!pip install -r requirements.txt



import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom as dicom
import os
from scipy import ndimage
import matplotlib.pyplot as plt
import cv2 as cv
import vtk
from vtk.util import numpy_support
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import torch
# take folder of a single lung and preprocess and segments the lungs



class preprocess:
    def __init__(self, input_folder, metadata_path = None, dilation_kernel_size = 2, dilation_iteration = 2):
        self.metadata_path = metadata_path
        self.dilation_kernel_size = dilation_kernel_size
        self.dilation_iteration = dilation_iteration
        self.input_folder = input_folder
        self.loaded_scans = self.load_scan(self.input_folder)
        self.pid = self.loaded_scans[1].PatientID
        self.scan_HU_values = self.get_pixels_hu(self.loaded_scans)
        self.resampled_3d_image, self.spacing = self.resample(self.scan_HU_values, self.loaded_scans, [1,1,1])
        self.segmented_lung_mask = self.segment_lung_mask(self.resampled_3d_image, True, dilation = '3d', kernel_size = self.dilation_kernel_size, iteration = self.dilation_iteration)
        self.segmented_grayscale_scan = self.hu_transform(self.resampled_3d_image, self.segmented_lung_mask, lung_window= [-1200, 600])
        self.label = self.load_metadata()
    
    def save_astensor(self, path_to_save):
        torch.save(torch.from_numpy(self.segmented_grayscale_scan), f'{path_to_save}{self.pid}.pt')

    def load_metadata(self):
        #path = path if path else self.metadata_path
        path =self.metadata_path
        metadata = pd.read_csv(path)
        metadata = metadata.query('rndgroup == 1 ')
        metadata = metadata[[ 'pid','lung_cancer', 'conflc', 'canc_rpt_link', 'cancyr' ]].query(f'pid == {self.pid}')['lung_cancer'].to_list()[0]


        
        return metadata


    def hu_transform(self, scan_HU_value, mask, lung_window = [-1200, 600]):
        
        mask = mask.astype(np.int16)
        
        scaled_scan = (scan_HU_value-lung_window[0])/(lung_window[1]-lung_window[0])
        scaled_scan[scaled_scan<0]=0
        scaled_scan[scaled_scan>1]=1

        trans_scaled_scan = (scaled_scan*255).astype('uint8')
        segmented_trans_scan = mask * trans_scaled_scan
        
        return segmented_trans_scan

    def visualize_intensity_histogram(self, array ):
        plt.hist(array.flatten(), bins=100, color='c')
        plt.xlabel("intensity range")
        plt.ylabel("Frequency")
        plt.show()

                

    def load_scan(self, path):
        slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        try:
            slice_thickness = np.abs(
                slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
        except:
            slice_thickness = np.abs(
                slices[0].SliceLocation - slices[1].SliceLocation)

        for s in slices:
            s.SliceThickness = slice_thickness

        return slices


    def get_pixels_hu(self, slices):
        image = np.stack([s.pixel_array for s in slices])
        # Convert to int16 (from sometimes int16),
        # should be possible as values should always be low enough (<32k)
        image = image.astype(np.int16)

        # Set outside-of-scan pixels to 0
        # The intercept is usually -1024, so air is approximately 0
        image[image == -2000] = 0

        # Convert to Hounsfield units (HU)
        for slice_number in range(len(slices)):

            intercept = slices[slice_number].RescaleIntercept
            slope = slices[slice_number].RescaleSlope

            if slope != 1:
                image[slice_number] = slope * image[slice_number].astype(np.float64)
                image[slice_number] = image[slice_number].astype(np.int16)

            image[slice_number] += np.int16(intercept)

        return np.array(image, dtype=np.int16)

    def resample(self, image, scan, new_spacing=[1, 1, 1]):
        # Determine current pixel spacing
        spacing = np.array([scan[0].SliceThickness] +
                           list(scan[0].PixelSpacing), dtype=np.float32)

        resize_factor = spacing / new_spacing
        new_real_shape = image.shape * resize_factor
        new_shape = np.round(new_real_shape)
        real_resize_factor = new_shape / image.shape
        new_spacing = spacing / real_resize_factor

        image = ndimage.zoom(image, real_resize_factor, mode='nearest')
        #print(f'new_real_shape = {new_real_shape}')
        second_resize = ([320, 410, 410]) / new_shape
        image = ndimage.zoom(image, second_resize, mode='nearest')

        return image, new_spacing


    def largest_label_volume(self, im, bg=-1):
        vals, counts = np.unique(im, return_counts=True)

        counts = counts[vals != bg]
        vals = vals[vals != bg]

        if len(counts) > 0:
            return vals[np.argmax(counts)]
        else:
            return None



    def dilate(self, img, kernel_size, iteration, three_d=False, shape = 'circular'):

        img = img.copy()
        if three_d:
            if shape == 'circular' :
                kernel = morphology.ball(kernel_size).astype(np.uint8)
            elif shape == 'square' :
                kernel = np.ones((kernel_size, kernel_size, kernel_size), np.uint8)
            img = ndimage.binary_dilation(img, kernel, iterations=iteration)
        else:
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            for i, slide in enumerate(img):
                img[i] = ndimage.binary_dilation(
                    slide, kernel, iterations=iteration)

        return img


    def segment_lung_mask(self, image, fill_lung_structures=True, dilation='3d', kernel_size=3, iteration=5):

        # 1 in dilate does 2d dilation and 2 does 3d
        # not actually binary, but 1 and 2.
        # 0 is treated as background, which we do not want
        binary_image = np.array(image > -320, dtype=np.int8)+1
        # plot_3d(binary_image - 1, 0, 'thresholding')
        labels = measure.label(binary_image)

        # Pick the pixel in the very corner to determine which label is air.
        #   Improvement: Pick multiple background labels from around the patient
        #   More resistant to "trays" on which the patient lays cutting the air
        #   around the person in half
        background_labels = labels[0:2, 0:2, 0:2]
        background_label = np.median(background_labels)

        # Fill the air around the person
        binary_image[background_label == labels] = 2

        # Method of filling the lung structures (that is superior to something like
        # morphological closing)
        if fill_lung_structures:
            # For every slice we determine the largest solid structure
            for i, axial_slice in enumerate(binary_image):
                axial_slice = axial_slice - 1
                labeling = measure.label(axial_slice)
                l_max = self.largest_label_volume(labeling, bg=0)

                if l_max is not None:  # This slice contains some lung
                    binary_image[i][labeling != l_max] = 1

        binary_image -= 1  # Make the image actual binary
        binary_image = 1-binary_image  # Invert it, lungs are now 1

        # Remove other air pockets insided body
        labels = measure.label(binary_image, background=0)
        l_max = self.largest_label_volume(labels, bg=0)
        if l_max is not None:  # There are air pockets
            binary_image[labels != l_max] = 0

        if dilation == '2d':
            binary_image = self.dilate(binary_image, kernel_size, iteration)
        elif dilation == '3d':
            binary_image = self.dilate(binary_image, kernel_size, iteration, True)

        return binary_image



def prepare_scans(cases_path, metadata_path):
    scans = []
    for case in cases_path: 
        scan = preprocess(case, metadata_path)
        scans.append(scan)
    return scans


def save_scans(cases_path, metadata_path, save_path):
    for case in cases_path: 
        scan = preprocess(case, metadata_path)
        scan.save_astensor(save_path)
    print("all the scans saved ! ")


def return_scan_path(path, sliceThreshold):
    scans_path = []
    for d in os.listdir(path):
        if os.path.isdir(f'{path}\{d}') :
            for dd in os.listdir(f'{path}\{d}'):
                if os.path.isdir(f'{path}\{d}\{dd}') :
                    for ddd in os.listdir(f'{path}\{d}\{dd}'):
                        if len(os.listdir(f'{path}\{d}\{dd}\{ddd}')) >= sliceThreshold: 
                            scans_path.append(f'{path}\{d}\{dd}\{ddd}')

    return scans_path


In [ ]:
# local computer
#INPUT_FOLDER = './dataset/manifest-1674660649243'

# lab 
INPUT_FOLDER = '../dataset/manifest-1674660649243'

NLST = f'{INPUT_FOLDER}/NLST flatten'

expFolder = f'{NLST}/100004/01-02-2001-NA-NLST-LSS-01308/1.000000-2OPATOAQUL4C410.22-06725'
cases_root_folder = f'{NLST}/100004/01-02-2001-NA-NLST-LSS-01308/'
meta_data = './dataset/metadata/participant_d040722.csv'
# an array including path of all the scan folders
cases_path = [cases_root_folder + fileName for fileName in os.listdir(cases_root_folder)]


positives_path = return_scan_path(r'c:\Users\Woody\Desktop\lung cancer\dataset\train_positive\manifest-1681052108196\NLST', 100)
negetives_path = return_scan_path(r'c:\Users\Woody\Desktop\lung cancer\dataset\train_negetive\manifest-1681053813358\NLST', 100)


path_to_positives = f'c:/Users/Woody/Desktop/lung cancer/dataset/train_positive/manifest-1681052108196/segmented/'
path_to_negetives = f'c:/Users/Woody/Desktop/lung cancer/dataset/train_negetive/manifest-1681053813358/segemented/'



save_scans(positives_path, meta_data, path_to_positives)
save_scans(negetives_path, meta_data, path_to_negetives)



#just for testing, scans is probably an array containing scans, cancerous and non cancerous combined 
#cancerous = list(filter(lambda x : bool(x.label) == True,  scans))
#noncancerous = list(filter(lambda x : bool(x.label) == False,  scans))


In [ ]:
class vtk_visualization:

    colors = vtk.vtkNamedColors()

    def __init__(self, structure, tissue_color = 'white', max_opacity = False, gradient_transfer = False):
        self.gradient_transfer = gradient_transfer
        self.max_opacity = max_opacity
        self.tissue_color = tissue_color
        self.structure = structure 
        self.imData = self.numpyToImageDate(structure)
        self.volumeMapper, self.volumeProperty = self.imDataToVolumeMapper(self.imData)
        self.vtkVolume = vtk.vtkVolume()
        self.volume = self.vMapperToVolume(self.vtkVolume, self.volumeMapper, self.volumeProperty)
        self.ren = self.renderVol(self.volume)

    def numpyToImageDate(self, numpyScan):
        scan = numpyScan.transpose(2,0,1)
        imData = vtk.vtkImageData()
        vtk_data = numpy_support.numpy_to_vtk(scan.ravel(order= 'F'), deep=True, array_type=vtk.VTK_DOUBLE)

        imData.SetDimensions(scan.shape)
        imData.SetSpacing([.1,.1,.1])
        imData.SetOrigin([0,0,0])
        imData.GetPointData().SetScalars(vtk_data)

        return imData

    #tcolor referes to tissue color
    #mopac referes to max_opacity
    #gt referes gradient_transfer
    def imDataToVolumeMapper(self, imData, tcolor =  self.tissue_color, mopac = self.max_opacity, gt = self.gradient_transfer):
        opacity = vtk.vtkPiecewiseFunction()
        if not bool(mopac) :
            
            opacity.AddPoint(0, 0)
            opacity.AddPoint(150, .05)
            opacity.AddPoint(180, 1)
            opacity.AddPoint(210, .1)
            opacity.AddPoint(255, .001)
        else:     
            opacity.AddPoint(0, 0)
            opacity.AddPoint(250, mopac)

        color = vtk.vtkColorTransferFunction()
        r, g, b = vtk_visualization.colors.GetColor3ub('black')
        color.AddRGBPoint(0, r, g, b)

        r, g, b = vtk_visualization.colors.GetColor3ub(self.tcolor)
        color.AddRGBPoint(255, r, g, b)
        '''
        r, g, b = vtk_visualization.colors.GetColor3ub('white')
        color.AddRGBPoint(180, r, g, b)
        
        r, g, b = vtk_visualization.colors.GetColor3ub('red')
        color.AddRGBPoint(254, r, g, b)
        '''
        '''        
        r, g, b = vtk_visualization.colors.GetColor3ub('blue')
        color.AddRGBPoint(255, r, g, b)
        '''

        funcOpacityGradient = vtk.vtkPiecewiseFunction()

        funcOpacityGradient.AddPoint(1,   0.0)
        funcOpacityGradient.AddPoint(5,   0.1)
        funcOpacityGradient.AddPoint(50,   1.0)



        volumeProperty = vtk.vtkVolumeProperty()
        volumeProperty.SetColor(color)
        volumeProperty.ShadeOff()
        if self.gradient_transfer :
            volumeProperty.SetGradientOpacity(funcOpacityGradient)
        volumeProperty.SetScalarOpacity(opacity)
        volumeProperty.SetInterpolationTypeToLinear()
        volumeProperty.SetIndependentComponents(2)


        volumeMapper = vtk.vtkOpenGLGPUVolumeRayCastMapper()
        volumeMapper.SetInputData(imData)
        #volumeMapper.SetBlendModeToAverageIntensity()
        return volumeMapper, volumeProperty 


    def vMapperToVolume(self, volume, mapper, property):
        volume.SetMapper(mapper)
        volume.SetProperty(property)
        return volume
# there should be another volume gotten from the above function.

    def renderVol(self, volume):

        ren = vtk.vtkRenderer()
        ren.AddVolume(volume)
        ren.SetBackground(vtk_visualization.colors.GetColor3ub('black'))
        return ren

    def visualize(self, ren):

        render_window = vtk.vtkRenderWindow()
        render_window.AddRenderer(ren)
        render_window.SetSize(400, 400)


        render_window_interactor = vtk.vtkRenderWindowInteractor()
        render_window_interactor.SetRenderWindow(render_window)
        render_window_interactor.Initialize()

        ren.ResetCamera()
        ren.GetActiveCamera().Zoom(1.5)

        render_window.Render()
        render_window_interactor.Start()
    # n_ refers to a new scan
    def return_new_volume(self, n_structure, tcolor, opac, grad_transfer):

        n_imData = self.numpyToImageDate(n_structure)
        n_volumeMapper, n_volumeProperty = self.imDataToVolumeMapper(n_imData, tcolor, opac, grad_transfer)
        n_vtkVolume = vtk.vtkVolume()
        n_volume = self.vMapperToVolume(n_vtkVolume, n_volumeMapper, n_volumeProperty)
        return n_volume
    
    def add_new_volume(self, structure, tcolor, opac, grad_transfer):
        
        new_volume = self.return_new_volume(self, structure, tcolor, opac, grad_transfer)
        self.ren.AddVolume()
        



In [ ]:

#this part as a whole should be removed soon. 

def gaussian_filter(sigma, size):
    diameter = int(round(size/2))
    x = np.arange(-diameter,diameter + 1,1)   # coordinate arrays -- make sure they contain 0!
    y = np.arange(-diameter,diameter + 1,1)
    z = np.arange(-diameter,diameter + 1,1)
    xx, yy, zz = np.meshgrid(x,y,z)
    kernel = np.exp(-(xx**2 + yy**2 + zz**2)/(2*sigma**2))
    
    normalized_kernel =   (((kernel - kernel.min() ) / (kernel.max() - kernel.min())) * 255).astype(np.int32)
    return normalized_kernel

part0 = gaussian_filter(15, 51)
part1 = gaussian_filter(5, 25)
part2 = gaussian_filter(20, 57)

gc = np.zeros((75, 75, 75))
gc[0:53, 0:53, 10:63] = part0
gc[6:31, 0:25, 2:27] = part1
gc[9:66, 5:62, 5:62] = part2




In [ ]:

'''
scan_vis = vtk_visualization(cancerous[0].segmented_grayscale_scan)
scan_vis.visualize(scan_vis.ren)


scan_vis = vtk_visualization(cancerous[1].segmented_grayscale_scan)
scan_vis.visualize(scan_vis.ren)


scan_vis = vtk_visualization(cancerous[2].segmented_grayscale_scan)
scan_vis.visualize(scan_vis.ren)


scan_vis = vtk_visualization(cancerous[3].segmented_grayscale_scan)
scan_vis.visualize(scan_vis.ren)

'''

#just for making up something for the project: 
#scan_vis = vtk_visualization(noncancerous[1].segmented_grayscale_scan)
#scan_vis.visualize(scan_vis.ren)



In [ ]:
#will be removed soon, jsut for making up somethign, from this part till hashtags

part0 = gaussian_filter(9, 49)
part1 = gaussian_filter(10, 35)
part2 = gaussian_filter(9, 25)

gc = np.zeros((50, 50, 50))
gc[0:49, 0:49, 0:49] = part0
gc[0:37, 0:37, 0:37] = part1
gc[9:34, 5:30, 5:30] = part2

conctenated = np.concatenate((gc, gc))

print(conctenated.shape)

print(cancerous[1].segmented_grayscale_scan.shape)


gradCam = np.zeros(cancerous[1].segmented_grayscale_scan.shape, dtype= np.int16)


#attention = ndimage.gaussian_filter(attention_base, 1 )

gradCam[140: 190, 215: 265, 60: 110] = gc

##########################################################################

'''
gradCam_vis = vtk_visualization(gradCam, 'red', .7, True)
#gradCam_vis.visualize(gradCam_vis.ren)

#gc = grad cam 

gc_mapper = gradCam_vis.volume
#AddVolume(gc_mapper)
scan_vis.ren.AddVolume(gc_mapper)
scan_vis.visualize(scan_vis.ren)



#gradCam_vis.visualize(gradCam_vis)



'''


#scan_vis.add_new_volume(gradCam, 'red', 0.7 , True)

In [ ]:
# visualizing intensity histogram for finding a threshold for transfer functions : 


intensities = scans[0].segmented_grayscale_scan.flatten()
scans[0].visualize_intensity_histogram(scans[0].segmented_grayscale_scan)


arr = np.delete(intensities, np.where(intensities == 0))
#arr = np.delete(arr, np.where(arr == 255))
scans[0].visualize_intensity_histogram(arr)



from skimage.filters import threshold_multiotsu

thresholds = threshold_multiotsu(arr,2)

print(thresholds)